In [7]:
import pandas as pd
import os

In [8]:
nbl = """
COG-N-623x
COG-N-590x
NCH-NB-1
COG-N-589x
COG-N-625x
COG-N-619x
COG-N-620x
COG-N-603x
""".strip().split('\n')

In [9]:
data_dir = '/home/jpfeil/treehouse/maris/pdx/data'

In [10]:
pth = os.path.join(data_dir, '2018-12-28-pdx-clinical-final-for-paper.txt')

pdx_clinical = pd.read_csv(pth, sep='\t')

# Only consider models that are part of the PPTC
pdx_clinical = pdx_clinical[pdx_clinical['RNA Part of PPTC'] == 'yes']

In [14]:
import collections

out = pdx_clinical[pdx_clinical['Model'].isin(nbl)]

out_count = collections.Counter(out['Histopathology'].values)

out_count

Counter({'MYCN Amp': 3, 'Non-MYCN Amp': 5})

In [15]:
mask = (~pdx_clinical['Model'].isin(nbl)) & (pdx_clinical['Histology-Detailed'] == 'Neuroblastoma')

_in = pdx_clinical[mask]

_in_count = collections.Counter(_in['Histopathology'].values)

_in_count

Counter({'MYCN Amp': 18,
         'MYCN Amp by pathology, not by genetics': 1,
         'Non-MYCN Amp': 6})

In [25]:
import numpy as np
from rpy2.robjects import r, Matrix, globalenv, pandas2ri
from rpy2.robjects.packages import importr
import rpy2.robjects.numpy2ri
import warnings 
warnings.filterwarnings("ignore")
    
rpy2.robjects.r['options'](warn=-1)
    
rpy2.robjects.numpy2ri.activate()

base = importr('base')
stats = importr('stats')

m = np.zeros((2, 2))

m[0, 0] = _in_count['Non-MYCN Amp']
m[0, 1] = _in_count['MYCN Amp'] + _in_count['MYCN Amp by pathology, not by genetics']

m[1, 0] = out_count['Non-MYCN Amp']
m[1, 1] = out_count['MYCN Amp']

print m

stats.fisher_test(m)[0][0]

[[ 6. 19.]
 [ 5.  3.]]


0.08249669623418239